In [1]:
from pyspark.sql import SparkSession
#import pandas as pd
import numpy as np
from pyspark.sql.functions import *
#import boto3
import io

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1596395738688_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
plyr=spark.read.format("csv").load("s3://kalange-european-soccer/ply.csv",header='true')
coun = spark.read.format("csv").load("s3://kalange-european-soccer/country.csv",header='true')
league = spark.read.format("csv").load("s3://kalange-european-soccer/league.csv",header='true')
#match = spark.read.format("csv").load("s3://kalange-european-soccer/match1.csv",header='true')
team = spark.read.format("csv").load("s3://kalange-european-soccer/team.csv",header='true')
team_attr = spark.read.format("csv").load("s3://kalange-european-soccer/team_attr.csv",header='true')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
plyr_attr= spark.read.format("csv").load("s3://kalange-european-soccer/ply_attr.csv",header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
xply=plyr_attr.select(plyr_attr.columns[:7])
yply=plyr_attr.select(['id','stamina','strength'])

xply=xply.alias('xply')
yply=yply.alias('yply')

df_plyr=xply.join(yply,xply.id==yply.id).select("xply.id","player_fifa_api_id","player_api_id","date","overall_rating", \
                                               "potential","stamina","strength")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_plyr.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+-------------+----+--------------+---------+-------+--------+
| id|player_fifa_api_id|player_api_id|date|overall_rating|potential|stamina|strength|
+---+------------------+-------------+----+--------------+---------+-------+--------+
|  0|                 0|            0|   0|             0|        0|      0|       0|
+---+------------------+-------------+----+--------------+---------+-------+--------+

In [13]:
plyr.select([count(when(isnan(c), c)).alias(c) for c in plyr.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+-------------+-----------+------------------+--------+------+------+
|_c0| id|player_api_id|player_name|player_fifa_api_id|birthday|height|weight|
+---+---+-------------+-----------+------------------+--------+------+------+
|  0|  0|            0|          0|                 0|       0|     0|     0|
+---+---+-------------+-----------+------------------+--------+------+------+

In [14]:
coun.select([count(when(isnan(c), c)).alias(c) for c in coun.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+----+
|_c0| id|name|
+---+---+----+
|  0|  0|   0|
+---+---+----+

In [15]:
match= spark.read.format("csv").load("s3://kalange-european-soccer/match.csv",header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
mat=match.select(['id','country_id','league_id','season','stage','date','match_api_id','home_team_api_id','away_team_api_id',\
                'home_team_goal','away_team_goal'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
mat.select([count(when(isnan(c), c)).alias(c) for c in mat.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----------+---------+------+-----+----+------------+----------------+----------------+--------------+--------------+
| id|country_id|league_id|season|stage|date|match_api_id|home_team_api_id|away_team_api_id|home_team_goal|away_team_goal|
+---+----------+---------+------+-----+----+------------+----------------+----------------+--------------+--------------+
|  0|         0|        0|     0|    0|   0|           0|               0|               0|             0|             0|
+---+----------+---------+------+-----+----+------------+----------------+----------------+--------------+--------------+